In [35]:
import pandas as pd # type: ignore
import plotly.express as px
import plotly.graph_objects as go 
import numpy as np # type: ignore
import datetime as dt


data =pd.read_csv(r"C:\Users\loges\Downloads\excel\online_retail.csv") 
#pd.set_option('display_max_cols',8)
#pd.set_option('display_max_rows',max)



In [ ]:
data['InvoiceDate']=pd.to_datetime(data['InvoiceDate'])
data['TotalAmount']=data['Quantity']*data['UnitPrice']
data

In [ ]:
reference_date=data['InvoiceDate'].max()+pd.Timedelta(days=1)
reference_date

In [ ]:
rfm=data.groupby('CustomerID').agg({'InvoiceDate':lambda x:(reference_date-x.max()).days,'InvoiceNo':'count','TotalAmount':'sum'})
rfm.rename(columns={'InvoiceDate':'Recency','InvoiceNo':'Frequncy','TotalAmount':'Value'},inplace=True)
rfm

In [ ]:
quantiles=rfm.quantile(q=[0.25,0.5,0.75])


#assign rfm scores
def Rscore(x,p,d):
    if p== 'Recency':
        if x<=d[p][0.25]:
            return 4
        elif x<=d[p][0.5]:
            return 3
        elif x<=d[p][0.75]:
            return 2
        else:
            return 1
    else:
        if x<=d[p][0.25]:
            return 1
        elif x<=d[p][0.5]:
            return 3
        elif x<=d[p][0.75]:
            return 2
        else:
            return 1

In [ ]:
rfm['R']= rfm['Recency'].apply(Rscore,args=('Recency',quantiles))
rfm['F']=rfm['Frequncy'].apply(Rscore,args=('Frequncy',quantiles))
rfm['V']=rfm['Value'].apply(Rscore,args=('Value',quantiles))
rfm

In [ ]:
rfm['CustomerSegment']=rfm['R'].astype(str)+rfm['F'].astype(str)+rfm["V"].astype(str)
rfm['RFM_Score']=rfm[['R','F','V']].sum(axis=1)
rfm

In [ ]:
def assign_segment(score):
    if score<5:
        return 'low-value'
    if score<9:
        return 'mid-value' 
    else :
        return "high-value"
rfm['RFM_SegmentLables']=rfm['RFM_Score'].apply(assign_segment)
rfm

In [ ]:
segmennt_counts=rfm['RFM_SegmentLables'].value_counts().reset_index()
segmennt_counts.columns=['CustomerSegment','count']
#segmennt_counts=segmennt_counts.sort_values('CustomerSegment')
segmennt_counts

In [ ]:
fig=px.bar(segmennt_counts,
           x='CustomerSegment',
           y='count',
           title='consumer distribution',
           labels={'CustomerSegment':'Customer Segment','count':'No of Customer'},
           color='CustomerSegment' ,
           color_discrete_sequence=px.colors.qualitative.Pastel1
           )
fig.show()

In [ ]:
def rfm_seg(segment):
    if segment<=9:
        return 'VIP\loyal'
    elif segment>=6:
        return 'potential loyal'
    elif segment>=5:
        return 'at risk cusomer'
    elif segment>=4:
        return "can't lose"
    elif segment>=3:
        return "lost"
    else: return 

In [21]:
rfm['Score_review']=rfm["RFM_Score"].apply(rfm_seg)
rfm.loc[rfm['RFM_Score']<=9]

,Recency,Frequncy,Value,R,F,V,CustomerSegment,RFM_Score,RFM_SegmentLables,Score_review
CustomerID,,,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3,low-value,VIP\loyal
12347.0,2,182,4310.00,4,1,1,411,6,mid-value,VIP\loyal
12348.0,75,31,1797.24,2,3,1,231,6,mid-value,VIP\loyal
12349.0,19,73,1757.55,3,2,1,321,6,mid-value,VIP\loyal
12350.0,310,17,334.40,1,1,3,113,5,mid-value,VIP\loyal
...,...,...,...,...,...,...,...,...,...,...
18280.0,278,10,180.60,1,1,1,111,3,low-value,VIP\loyal
18281.0,181,7,80.82,1,1,1,111,3,low-value,VIP\loyal
18282.0,8,13,176.60,4,1,1,411,6,mid-value,VIP\loyal


In [ ]:
segment_product_counts=rfm.groupby(['CustomerSegment','RFM_SegmentLables']).size().reset_index(name="count")
segment_product_counts

In [22]:
tree_map=px.treemap(segment_product_counts,
                    path=["RFM_SegmentLables","CustomerSegment"],
                    values='count',
                    color_discrete_sequence=px.colors.qualitative.Pastel,
                    color='RFM_SegmentLables',
                    title="RFM segment lables by value")
tree_map.show()

In [24]:
vip_segment=rfm[rfm["Score_review"]=='VIP\loyal']
fig=go.Figure()
fig.add_trace(go.Box(y=vip_segment['Recency'],name='Recency'))
fig.add_trace(go.Box(y=vip_segment['Frequncy'],name='Frequncy'))
fig.add_trace(go.Box(y=vip_segment['Value'],name='Value'))

<>:1: SyntaxWarning:

invalid escape sequence '\l'

<>:1: SyntaxWarning:

invalid escape sequence '\l'

C:\Users\loges\AppData\Local\Temp\ipykernel_28660\948993095.py:1: SyntaxWarning:

invalid escape sequence '\l'



In [30]:
correlation_matrix=vip_segment[['R','F','V']].corr

In [36]:



heatmap = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='Viridis'
))

# Update layout for better presentation
heatmap.update_layout(
    title='Correlation Matrix Heatmap',
    xaxis_nticks=36,
    yaxis_nticks=36
)

# Show plot
heatmap.show()

AttributeError: 'function' object has no attribute 'values'